In [27]:
import requests
from urllib.parse import urlencode, parse_qs
from dotenv import load_dotenv
import os

load_dotenv()

True

In [28]:
APP_ID = os.getenv('META_APP_ID')
APP_SECRET = os.getenv('META_APP_SECRET')
REDIRECT_URI = os.getenv('REDIRECT_URI')
SCOPES = 'instagram_basic, instagram_manage_insights'
GRAPH_API_VERSION = 'v20.0'

# Gerar URL de autorização 

In [29]:
def generate_auth_url() -> str: 
    params = {
        'client_id':APP_ID,
        'redirect_uri':REDIRECT_URI,
        'scope': SCOPES,
        'response_type': 'code'
    }
    auth_url = f'https://www.facebook.com/{GRAPH_API_VERSION}/dialog/oauth?{urlencode(params)}'
    print(f'URL de Autorização: {auth_url}')
    return auth_url

# Receber code

In [30]:
url = generate_auth_url()


URL de Autorização: https://www.facebook.com/v20.0/dialog/oauth?client_id=859402103106794&redirect_uri=https%3A%2F%2Fsocialdatalab.vercel.app&scope=instagram_basic%2C+instagram_manage_insights&response_type=code


1. Copie e cole a URL acima no navegador.
2. Faça login com Facebook/Instagram (conta Business/Creator ligada a Page).
3. Aprove as permissões.
4. Será redirecionado para REDIRECT_URI com ?code=... no URL.
5. Copie o 'code' da query string (ex.: tudo após ?code= até o final).

# Trocar por short-lived token

In [31]:
def code_to_short_lived_token(code:str) -> str:
    params = {
        'client_id': APP_ID,
        'client_secret': APP_SECRET,
        'redirect_uri': REDIRECT_URI,
        'code': code
    }
    response = requests.post(f'https://graph.facebook.com/{GRAPH_API_VERSION}/oauth/access_token', params=params)
    if response.status_code == 200:
        data = response.json()
        print(f'Short-Lived Token: {data["access_token"]}')
        print(f'User ID: {data.get("user_id")}')
        print(f'Granted Scopes: {data.get("granted_scopes")}')
        return data['access_token']
    
    print(f'Erro: {response.text}')
    return None

In [32]:
# https://socialdatalab.vercel.app/?code=AQBBB6mVTgfb8sX_VUG7UOS0azaudZebf0XfTaMwfcA1Ww6bkrVH9U5eJXybNBGDA2El0fXgKHVjJPNyONSXco6RFcoDlOf6cR9DqFqXwlkQsU9_-xW6OyRBixqIfl3nXqR0uimQyXDu8JGHbqxl-3GXKUMqcJjmdaSFOFrC9kgV3iOEp0x96AN9irILqST-67C70PvDNB9L2fCTuiJ3RAvviUD0jnoxzSQ8QtWsjz-VXyZNB__u4uFr6c2q4pESLk206B-Ahxh-7ePAxXg1uS5kMv7DpFqv1wRwSOcvDnvTNnkQItUyObQYz6-Tmn2y9X4g9MbtYLTqz8neE80RXSpgnn4fFOJfKn3Tp-EhCIzP6ZKgznQR0JyZcfE-HKI_fw-kwHaQzzorjoostnfWjx0-rFQqvUvcS40XpkfozStcdjuMqbCZoEcDvCt8CARUB53IiTU2dNfWNaGy-rtue00O#_=_
code = 'AQBBB6mVTgfb8sX_VUG7UOS0azaudZebf0XfTaMwfcA1Ww6bkrVH9U5eJXybNBGDA2El0fXgKHVjJPNyONSXco6RFcoDlOf6cR9DqFqXwlkQsU9_-xW6OyRBixqIfl3nXqR0uimQyXDu8JGHbqxl-3GXKUMqcJjmdaSFOFrC9kgV3iOEp0x96AN9irILqST-67C70PvDNB9L2fCTuiJ3RAvviUD0jnoxzSQ8QtWsjz-VXyZNB__u4uFr6c2q4pESLk206B-Ahxh-7ePAxXg1uS5kMv7DpFqv1wRwSOcvDnvTNnkQItUyObQYz6-Tmn2y9X4g9MbtYLTqz8neE80RXSpgnn4fFOJfKn3Tp-EhCIzP6ZKgznQR0JyZcfE-HKI_fw-kwHaQzzorjoostnfWjx0-rFQqvUvcS40XpkfozStcdjuMqbCZoEcDvCt8CARUB53IiTU2dNfWNaGy-rtue00O'
short_lived_token = code_to_short_lived_token(code)

Erro: {"error":{"message":"Error validating verification code. Please make sure your redirect_uri is identical to the one you used in the OAuth dialog request","type":"OAuthException","code":100,"error_subcode":36008,"fbtrace_id":"AtVZhTMcebDjTspcSE2xEmX"}}


# Trocar por long-lived token

In [ ]:
def short_to_long_lived_token(short_tk:str):
    params = {
        'grant_type': 'fb_exchange_token',
        'client_id': APP_ID,
        'client_secret': APP_SECRET,
        'fb_exchange_token': short_tk  
    }
    response = requests.get(f'https://graph.facebook.com/{GRAPH_API_VERSION}/oauth/access_token', params=params)
    if response.status_code == 200:
        data = response.json
        print(f'Long-Lived Token: {data["access_token"]}')
        print(f'Expires In: {data["expires_in"]} seconds (~60 days)')
        return data['access_token']
    
    print(f'Erro: {response.text}')
    return None

In [ ]:
long_lived_token = short_to_long_lived_token(short_lived_token)